# Labo 1 - ETL met Python (pETL)

pETL is een python package specifiek voor ETL taken.  Meer info: <br />
Extract & load: http://petl.readthedocs.io/en/latest/io.html <br/>
Transform: http://petl.readthedocs.io/en/latest/transform.html 

In [2]:
import petl as etl

## Extract

#### Extract JSON bestand en weergeven

In [3]:
filename = 'drinks.json'

In [4]:
t1 = etl.fromjson(filename)
t1.display(10)

country,continent,total_pure_alcohol,servings
Afghanistan,AS,0.0 Litres,"[beer:0, spirit:0, wine:0]"
Albania,EU,4.9 Litres,"[beer:89, spirit:132, wine:54]"
Algeria,AF,0.7 Litres,"[beer:25, spirit:0, wine:14]"
Andorra,EU,12.4 Litres,"[beer:245, spirit:138, wine:312]"
Angola,AF,5.9 Litres,"[beer:217, spirit:57, wine:45]"
Antigua & Barbuda,None,4.9 Litres,"[beer:102, spirit:128, wine:45]"
Argentina,SA,8.3 Litres,"[beer:193, spirit:25, wine:221]"
Armenia,EU,3.8 Litres,"[beer:21, spirit:179, wine:11]"
Australia,OC,10.4 Litres,"[beer:261, spirit:72, wine:212]"
Austria,EU,9.7 Litres,"[beer:279, spirit:75, wine:191]"


Een voordeel van het gebruik van Petl is de lage memory consumption, daarnaast kan Petl de data direct uit zip bestanden inlezen!

In [5]:
source = etl.sources.ZipSource('drinks.zip', 'drinks.json')
etl.fromjson(source).display()

country,continent,total_pure_alcohol,servings
Afghanistan,AS,0.0 Litres,"[beer:0, spirit:0, wine:0]"
Albania,EU,4.9 Litres,"[beer:89, spirit:132, wine:54]"
Algeria,AF,0.7 Litres,"[beer:25, spirit:0, wine:14]"
Andorra,EU,12.4 Litres,"[beer:245, spirit:138, wine:312]"
Angola,AF,5.9 Litres,"[beer:217, spirit:57, wine:45]"


#### Het aantal records bekijken

In [6]:
t1.nrows()

193

#### Alle kolommen inlezen

In [7]:
fields = t1.fieldnames()
print(fields)

('country', 'continent', 'total_pure_alcohol', 'servings')


#### Benaderen van een specifieke waarde

In [8]:
print(t1[2][1])

EU


Of met behulp van slicen

In [9]:
# print specifieke waarde m.b.v. slicen
print(t1['total_pure_alcohol'][1])

4.9 Litres


#### Types van kolommen ophalen

In [10]:
# for lus om alle velden op te halen en bijhorend de types
for f in fields:
    print(f, '\t', t1.typecounter(f))

country 	 Counter({'str': 193})
continent 	 Counter({'str': 170, 'NoneType': 23})
total_pure_alcohol 	 Counter({'str': 193})
servings 	 Counter({'str': 193})


Een aantal problemen die in onze data voorkomen: 
<ul>
    <li>total alcohol is een string en geen getal (integer)</li>
    <li>servings is een unicode string en geen list of dictionary</li>
    <li>continent kan lege waarden bevatten</li>
</ul>  

## Transform 

#### Functie definiëren om total_pure_alcohol te converteren naar float 

In [11]:
def to_float(text):
    return float(text[:-7])
print(to_float('4.9 Litres'))

4.9


#### Functie uitvoeren op elke rij

In [12]:
t2 = t1.convert('total_pure_alcohol', to_float)

In [13]:
print(t2.typecounter('total_pure_alcohol'))
t2.display(10)

Counter({'float': 193})


country,continent,total_pure_alcohol,servings
Afghanistan,AS,0.0,"[beer:0, spirit:0, wine:0]"
Albania,EU,4.9,"[beer:89, spirit:132, wine:54]"
Algeria,AF,0.7,"[beer:25, spirit:0, wine:14]"
Andorra,EU,12.4,"[beer:245, spirit:138, wine:312]"
Angola,AF,5.9,"[beer:217, spirit:57, wine:45]"
Antigua & Barbuda,None,4.9,"[beer:102, spirit:128, wine:45]"
Argentina,SA,8.3,"[beer:193, spirit:25, wine:221]"
Armenia,EU,3.8,"[beer:21, spirit:179, wine:11]"
Australia,OC,10.4,"[beer:261, spirit:72, wine:212]"
Austria,EU,9.7,"[beer:279, spirit:75, wine:191]"


#### Editeren van het servings veld 

Wij starten met een conversie van string -> dictionary <br/>
Het dictionary type is een bruikbare datastructuur voor de volgende stappen, wij kunnen het benaderen via "keys" en "values"

In [14]:
# controleren van de inhoud van de servings kolom op een willekeurige waarde
sample = t2['servings'][5]
print(sample)
print(type(sample))

[beer:102, spirit:128, wine:45]
<class 'str'>


In [15]:
def to_dict(text):
    if text[0]=='[' and text[-1]==']':
        items = text[1:-1].split(', ') #values -> list
    else:
        return None
    output = {}
    for i in items:
        key, value = i.split(':')
        output[key] = int(value)
    return output

print(to_dict(sample))
print(type(to_dict(sample)))

{'beer': 102, 'spirit': 128, 'wine': 45}
<class 'dict'>


In [16]:
# volledige lijst converteren
t3 = t2.convert('servings', to_dict)
t3.typecounter('servings')

Counter({'dict': 193})

#### Unpack servings (converteren van iterable object naar aantal velden) 

In [17]:
t4 = t3.unpackdict('servings')
t4.display(5)

country,continent,total_pure_alcohol,beer,spirit,wine
Afghanistan,AS,0.0,0,0,0
Albania,EU,4.9,89,132,54
Algeria,AF,0.7,25,0,14
Andorra,EU,12.4,245,138,312
Angola,AF,5.9,217,57,45


Meer info over de "unpacking": http://petl.readthedocs.io/en/latest/transform.html#unpacking-compound-values

#### Toevoegen van een nieuw veld 'total price' 

In [18]:
def total_serving(row):
    return row['beer']+row['spirit']+row['wine']

In [19]:
t5 = t4.addfield('total_serving', total_serving)
t5.display(10)

country,continent,total_pure_alcohol,beer,spirit,wine,total_serving
Afghanistan,AS,0.0,0,0,0,0
Albania,EU,4.9,89,132,54,275
Algeria,AF,0.7,25,0,14,39
Andorra,EU,12.4,245,138,312,695
Angola,AF,5.9,217,57,45,319
Antigua & Barbuda,None,4.9,102,128,45,275
Argentina,SA,8.3,193,25,221,439
Armenia,EU,3.8,21,179,11,211
Australia,OC,10.4,261,72,212,545
Austria,EU,9.7,279,75,191,545


#### Knippen van een deeltabel uit de volledige countries tabel

In [20]:
out_countries = t5.cut(['country', 'continent', 'total_pure_alcohol', 'total_serving'])
out_countries.display()

country,continent,total_pure_alcohol,total_serving
Afghanistan,AS,0.0,0
Albania,EU,4.9,275
Algeria,AF,0.7,39
Andorra,EU,12.4,695
Angola,AF,5.9,319


#### Converteren van een rij in meerdere rijen 

In [21]:
# knippen van de noodzakelijke kolommen
t6 = t5.cut(['country', 'beer', 'wine', 'spirit'])
t6.display()

country,beer,wine,spirit
Afghanistan,0,0,0
Albania,89,54,132
Algeria,25,14,0
Andorra,245,312,138
Angola,217,45,57


In [22]:
# melting (row -> rows)
t7 = t6.melt(key=['country'])
t7.display(6)

country,variable,value
Afghanistan,beer,0
Afghanistan,wine,0
Afghanistan,spirit,0
Albania,beer,89
Albania,wine,54
Albania,spirit,132


In [23]:
# filteren van nulwaarden en toevoegen van row_numbers
t8 = t7.selectisnot('value',0)
t9 = t8.addrownumbers()
t9.display()

row,country,variable,value
1,Albania,beer,89
2,Albania,wine,54
3,Albania,spirit,132
4,Algeria,beer,25
5,Algeria,wine,14


In [24]:
# hernoemen van kolommen
out_servings = t9.rename({'row':'s_id', 'variable':'type', 'value':'servings'})
out_servings.display()

s_id,country,type,servings
1,Albania,beer,89
2,Albania,wine,54
3,Albania,spirit,132
4,Algeria,beer,25
5,Algeria,wine,14


In [27]:
# controle bij joinen van tabellen in logische uitvoer
out_countries.join(out_servings, #rechtertabel
                   lkey='country', rkey='country', #joinen van gelijke kolommen
                   lprefix='c_', rprefix='serv_') #prefixen van kolommen

c_country,c_continent,c_total_pure_alcohol,c_total_serving,serv_s_id,serv_type,serv_servings
Albania,EU,4.9,275,1,beer,89
Albania,EU,4.9,275,2,wine,54
Albania,EU,4.9,275,3,spirit,132
Algeria,AF,0.7,39,4,beer,25
Algeria,AF,0.7,39,5,wine,14


De uitgebreide Transform library kan gecombineerd worden met meer interne en externe functies. Het is aangewezen om de documentatie te bekijken van de volgende acties die mogelijk zijn: <br/>
<ul>
    <li>Cat/Stack</li>
    <li>Select family</li>
    <li>Sort</li>
    <li>Join/Leftjoin/Rightjoin/Outerjoin/Crossjoin</li>
    <li>Unjoin</li>
    <li>Unique/Duplicates/Distinct</li>
    <li>Aggregate</li>
</ul>

In [28]:
# Toepassing van de aggregate functie om het maximum aantal servings te vinden per continent
t5.display(5)
t5.aggregate('continent', max,'total_serving').displayall()

country,continent,total_pure_alcohol,beer,spirit,wine,total_serving
Afghanistan,AS,0.0,0,0,0,0
Albania,EU,4.9,89,132,54,275
Algeria,AF,0.7,25,0,14,39
Andorra,EU,12.4,245,138,312,695
Angola,AF,5.9,217,57,45,319


continent,value
None,665
AF,504
AS,646
EU,695
OC,545
SA,439


## Load 

Wij kunnen de output bewaren op meerdere manieren. Wij starten met een uitvoer -> CSV

In [29]:
# countries -> csv
out_countries.tocsv('out_countries.csv')
# servings -> csv
out_servings.tocsv('out_servings.csv')


Vervolgens werken wij met de MySQL cursor om de data te laden

In [32]:
import MySQLdb as mdb

In [33]:
user = 'root'
passwd = 'admin'

In [34]:
con = mdb.connect(host='127.0.0.1', user=user, passwd=passwd)
cur = con.cursor()

Maken en gebruiken van het db schema

In [35]:
cur.execute('DROP SCHEMA IF EXISTS drinks')
cur.execute('CREATE SCHEMA IF NOT EXISTS drinks')
cur.execute('USE drinks')
cur.execute('SET SQL_MODE=ANSI_QUOTES') #belangrijk voor petl

0

In [36]:
out_countries.todb(cur,'countries',schema='drinks',create='True')

In [37]:
out_servings.todb(cur,'serving',schema='drinks',create='True')

Toevoegen van data aan de bestaande tabellen

In [39]:
addition = [{'country':'Neverland', 'continent':None, 'total_pure_alcohol':0, 'total_serving':0}]

In [40]:
add_table = etl.fromdicts(addition) #creatie van table object uit een dictionary
add_table.display(10)

country,continent,total_pure_alcohol,total_serving
Neverland,None,0,0


In [41]:
add_table.appenddb(cur, 'countries', schema='drinks', commit=True)

Opzoeken van data via een select statement

In [42]:
cur.execute(""" SELECT * FROM drinks.countries WHERE country like 'N%' ;""")

12

In [43]:
print(cur.fetchall())

(('North Korea', 'AS', 0.0, 0), ('Namibia', 'AF', 6.8, 380), ('Nauru', 'OC', 1.0, 57), ('Nepal', 'AS', 0.2, 11), ('Netherlands', 'EU', 9.4, 529), ('New Zealand', 'OC', 9.3, 457), ('Nicaragua', None, 3.5, 197), ('Niger', 'AF', 0.1, 6), ('Nigeria', 'AF', 9.1, 49), ('Niue', 'OC', 7.0, 395), ('Norway', 'EU', 6.7, 369), ('Neverland', None, 0.0, 0))
